In [ ]:
from collections import OrderedDict
from glob import glob
import numpy as np
import pandas as pd
from os.path import join
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import xarray as xr

import torch
import torch.nn as nn
from torch import Tensor
from torchmetrics.utilities.checks import _check_same_shape
from torchmetrics import Metric
import pytorch_lightning as pl
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
import pylab as plt
import matplotlib

In [ ]:
import sys
sys.path.insert(0, "../pismemulator/")

from nnemulator import PISMDataset

In [ ]:
dataset = PISMDataset(data_dir="../data/speeds_v2/",
                      samples_file="../data/samples/velocity_calibration_samples_100.csv",
                      target_file="../data/observed_speeds/greenland_obs_g1800m.nc",
                     target_corr_threshold=25)
            

In [ ]:
cd ../speedemulator/

In [21]:
run ../speedemulator/sample_posterior.py --emulator_dir ../speedemulator/2021_11_gris_2km_cpu --data_dir ../data/speeds_v2/ --target_file ../data/observed_speeds/greenland_obs_g1800m.nc

/Users/andy/Library/Python/3.9/lib/python/site-packages/pismemulator/nnemulator.py:416: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  samples = pd.read_csv(


The following simulations are missing:
   [337, 595, 539, 542]
  ... adjusting priors
  Loading data sets...


996it [00:45, 21.94it/s]


(array([  13,   13,   14, ..., 1487, 1487, 1487]), array([412, 413, 405, ..., 518, 519, 520]))


/Users/andy/Library/Python/3.9/lib/python/site-packages/pismemulator/nnemulator.py:471: RuntimeWarning: divide by zero encountered in log10
  response = np.log10(response)


***********************************************
***********************************************
Finding MAP point
***********************************************
***********************************************


IndexError: too many indices for tensor of dimension 1

In [6]:
data_dir="../data/speeds_v2/"
samples_file="../data/samples/velocity_calibration_samples_100.csv"
target_file="../data/observed_speeds/greenland_obs_g1800m.nc"
target_var="velsurf_mag"
target_corr_threshold=25.0
target_corr_var="thickness"
target_error_var="velsurf_mag_error"
training_var="velsurf_mag"
thinning_factor=1
normalize_x=True
log_y=True
threshold=100e3
epsilon=0
return_numpy=False


In [7]:
A = torch.randn(2, 2, dtype=torch.complex128)

In [8]:
A

tensor([[ 0.1921+0.0062j,  0.6773+0.8533j],
        [-0.9775+0.3516j, -0.1907+0.4130j]], dtype=torch.complex128)

In [11]:
B = torch.randn(2, 2, dtype=torch.float)

In [12]:
B

tensor([[-1.2980, -0.0163],
        [-1.6737, -0.4782]])

In [15]:
U, V = torch.linalg.eig(B)

In [4]:
import torch
import torch.nn as nn
from torch import Tensor
from torchmetrics.utilities.checks import _check_same_shape
from torchmetrics import Metric
import pytorch_lightning as pl
from collections import OrderedDict


In [41]:
n_hidden = [128, 256, 256, 128]
n_layers = 5
n = 0

models = []
for  n in range(n_layers - 2):
    models.append(nn.Sequential(
        OrderedDict(
            [
                ("Linear", nn.Linear(n_hidden[n], n_hidden[n+1])),
                ("LayerNorm", nn.LayerNorm(n_hidden)),
                ("Dropout", nn.Dropout(p=0.1)),
            ]
        ))
                 )

In [43]:
nn.ModuleList[models]

TypeError: 'type' object is not subscriptable